In [1]:
#!pip install xlsxwriter

## Imports

In [2]:
# Read stocks
import yfinance as yf

# Access DataFrame
import pandas as pd

## Sector Configuration

In [3]:
# This is the sector we are interested, for example, finsvcs, energy etc.
SECTOR_CODE = 'finsvcs'

## Other Configurations

In [4]:
# Output will go to the out/sector_code.xlsx
OUTPUT = f'out/{SECTOR_CODE}.xlsx'
METRICS = ['ROE','Yield','DIV Growth','Payout Ratio','P/E Ratio']

# Load tickers
with open(f'symbols/{SECTOR_CODE}.txt') as file:
    data = file.read()
SYMBOLS = data.split()

# Load all the rules as a DF
RULES_DF = pd.read_csv('config/rules.csv', sep='|')

## Collect Metrics data

In [5]:
# DF to collect exclusions
key_metrics_ex_df = pd.DataFrame()
# Initialize list with empty dictionaries. This collects raw metrics data for each metric
raw_metrics = [{} for sub in range(len(METRICS))]
# Placeholder to collect company names; symbol -> long name
comp_name_dict = {}

for symbol in SYMBOLS:
    ticker = yf.Ticker(symbol)

    # Company name
    comp_name_dict[symbol] = ticker.info['longName']
    
    #1 Return on equity
    if 'returnOnEquity' in ticker.info:
        raw_metrics[0][symbol] = round(ticker.info['returnOnEquity'] * 100,2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[0], 'Reason':'Missing Return on equity'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    #2 Yield
    if 'dividendYield' in ticker.info:
        raw_metrics[1][symbol] = round(ticker.info['dividendYield'] * 100, 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[1], 'Reason':'Missing Dividend Yield'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    #3 Dividend growth forecast
    if 'dividendYield' in ticker.info and 'trailingAnnualDividendYield' in ticker.info:
        fwd_div_yield = ticker.info['dividendYield']
        trailing_div_yield = ticker.info['trailingAnnualDividendYield']
        if trailing_div_yield > 0:
            raw_metrics[2][symbol] = round((((fwd_div_yield/trailing_div_yield) - 1) * 100), 2)
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[2], 'Reason':'Trailing Dividend Yield is negative'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
            
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[2], 'Reason':'Unable to calculate Dividend growth forecast'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    #4 Payout ratio
    if 'payoutRatio' in ticker.info:
        raw_metrics[3][symbol] = round(ticker.info['payoutRatio'] * 100, 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[3], 'Reason':'Missing Payout ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    #5 Price to earnings ratio
    if 'forwardPE' in ticker.info:
        raw_metrics[4][symbol] = round(ticker.info['forwardPE'], 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[4], 'Reason':'Missing P/E ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

## Create a DF from raw metrics

In [6]:
# DF to collect value key metrics for all the tickers
key_metrics_df = pd.DataFrame()

for idx in range(len(METRICS)):
    key_metrics_df[METRICS[idx]] = raw_metrics[idx]

# Add the ticker name as a column
key_metrics_df['Name'] = comp_name_dict

## Apply Rules

In [8]:
# List to collect rules results
rules = []
# Rule #1 - Return on equity greater than 10%
rules.append(key_metrics_df['ROE'] > 10)
# Rule #2 - Yield greater than 4.0%
rules.append(key_metrics_df['Yield'] > 4.0)
# Rule #3 - Dividend growth forecast to increase
rules.append(key_metrics_df['DIV Growth'] > 0)
# Rule #4 - Payout ratio less than 75%
rules.append(key_metrics_df['Payout Ratio'] < 75)
# Rule #5 - Price to earnings ratio less than 20x
rules.append(key_metrics_df['P/E Ratio'] < 20)

## Utility method to create an Excel Rules sheet

In [9]:
# Create a Rules sheet
# writer- Excel writer
# id - rules id (starting with 0)
# header_format - formatting for the header
# desc_format - formatter for description cell
def create_rule_sheet(writer, id, header_format, desc_format):
    workbook  = writer.book
    # Create a worksheet
    sheet_name = f'Rule {id+1}'
    worksheet = workbook.add_worksheet(sheet_name)
    worksheet.write(1, 5, 'Description')
    # ID is starting with 1
    worksheet.write(1, 6, RULES_DF.iloc[id]['Description'], desc_format)
    
    # Write the column headers with the defined format.
    for idx, value in enumerate(METRICS):
        worksheet.write(3, idx+1, value, header_format)

    # Write the table contents without headers, starting row 4
    key_metrics_df[rules[id]].to_excel(writer, sheet_name=sheet_name, startrow=4, header=False)
    worksheet.autofit()

## Generate the Excel output

In [10]:
with pd.ExcelWriter(OUTPUT, engine='xlsxwriter') as writer:
    workbook  = writer.book
    
    # The format for table headers
    header_format = workbook.add_format(
        {'text_wrap': True, 'valign': 'top', 'fg_color': '#D7E4BC', 'border': 1}
    )
    # The format for the description
    desc_format = workbook.add_format(
        {'border': 1, 'fg_color': '#D6EAF8'}
    )

    # Outcome Worksheet ----------------------------------------------------------
    sheet_name = 'Outcome'
    worksheet = workbook.add_worksheet(sheet_name)        
    
    # Write the column headers with the defined format.
    for idx, value in enumerate(METRICS):
        worksheet.write(1, idx+1, value, header_format)

    # Calculate the outcome by applying indiviudal rule outcomes. Start with Rule 1
    outcome = rules[0]
    for rule in rules[1:]:
        outcome = outcome & rule
    # Write the table contents without headers, starting row 2
    key_metrics_df[outcome].to_excel(writer, sheet_name=sheet_name, startrow=2, header=False)
    worksheet.autofit()

    # Rules id worksheets, 1,2, 3, 4 and 5 ---------------------------------------
    for idx in range(len(rules)):
        create_rule_sheet(writer, idx, header_format, desc_format)

    # Rules Matrix worksheet -----------------------------------------------------
    sheet_name = 'Rules Matrix'
    worksheet = workbook.add_worksheet(sheet_name)

    # Holds data for Rules matrix
    matrix_dict = {}
    for idx in range(len(rules)):
        matrix_dict[f'Rule {idx+1}'] = rules[idx]
    # Rules Matrix header
    for idx, key in enumerate(matrix_dict):
        worksheet.write(1, idx+1, key, header_format)

    # Rules Matrix table details (without the header)
    rules_matrix_df = pd.DataFrame(matrix_dict)
    # Add the ticker name as a column
    rules_matrix_df['Name'] = comp_name_dict
    rules_matrix_df.to_excel(writer, sheet_name=sheet_name, startrow=2, header=False)
    worksheet.autofit()
    
    # Rules worksheet --------------------------------------------------------------
    worksheet = workbook.add_worksheet('Rules')        
    for idx, value in enumerate(RULES_DF.columns.values):
        worksheet.write(1, idx, value, header_format)

    # Cells are text wrapped
    cell_format = workbook.add_format({'text_wrap': True})
    for idx, row in RULES_DF.iterrows():
        for col, value in enumerate(RULES_DF.columns.values):
            worksheet.write(idx+2, col, row[value], cell_format)

    # Increase the column widths to accomodate text
    worksheet.set_column(1, 1, 30)
    worksheet.set_column(2, 2, 130)
    
    # Increase the row height
    for index in range(5):
        worksheet.set_row(index+2, 30)

In [11]:
key_metrics_ex_df

,Symbol,Metric,Reason
0,AFI.AX,P/E Ratio,Missing P/E ratio
1,JHG.AX,P/E Ratio,Missing P/E ratio
2,ARG.AX,P/E Ratio,Missing P/E ratio
3,AMP.AX,Payout Ratio,Missing Payout ratio
4,BFL.AX,DIV Growth,Trailing Dividend Yield is negative
5,BFL.AX,P/E Ratio,Missing P/E ratio
